# **Isochrone Mapper - version 1 (Begginer)**
### **Goal:**
Create a simple tool which shows the areas reachable by car in a given time, from a location (maybe just the UK)<BR><BR>
**Tech stack:**
* Jupter - exploratory file
* folumn - plotting map
* OpenRouteService (ORS)- routing/isochrone API
* StreamLit

In [1]:
import requests 
import folium
from shapely.geometry import shape
import os 
from dotenv import load_dotenv
import folium

In [2]:
# load the .env file 
load_dotenv()

ORS_API_KEY = None

# Get the API key from the .env file
ORS_API_KEY = os.getenv("ORS_API_KEY")


if not ORS_API_KEY:
    raise ValueError("ORS_API_KEY not found in environment")



In [3]:
# Example co-ordinates
# Edgware lat long
lon_edgware = -0.278134
lat_edgware = 51.616806

In [4]:
def get_isochrone(lon, lat, minutes=30, profile="driving-car"):
    url = f"https://api.openrouteservice.org/v2/isochrones/{profile}" # OpenRouteService Isochrone API url

    # auth and return types
    headers = {
        "Authorization": ORS_API_KEY,
        "Content-Type": "application/json"
    }

    # body/parameters for the isochrone api
    body = {
        "locations": [[lon, lat]],      # can include more than one location in one call?
        "range": [minutes * 60],        # convert the minutes input into seconds
        "units": "m",
        "attributes": ["area", "reachfactor"]
    }


    response = requests.post(url=url, json=body, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None
    
    return response.json() # returns GeoJSON-style dict

In [5]:
# Create the GeoJSON dict (isochrone coordinates)
isochrone_Edgware = get_isochrone(lon_edgware, lat_edgware)

In [6]:
isochrone_Edgware['features'][0]['properties']['center']

[-0.27816788724692465, 51.61679987910633]

### Folium

In [7]:

FOLIUM_MARKER_COLORS = {
    "red": "#FF0000",
    "darkred": "#8B0000",
    "lightred": "#FF7F7F",
    "orange": "#FFA500",
    "beige": "#F5F5DC",
    "green": "#008000",
    "darkgreen": "#006400",
    "lightgreen": "#90EE90",
    "blue": "#2A81CB",   # Folium's "blue"
    "darkblue": "#00008B",
    "lightblue": "#ADD8E6",
    "purple": "#800080",
    "darkpurple": "#551A8B",
    "pink": "#FFC0CB",
    "cadetblue": "#5F9EA0",
    "white": "#FFFFFF",
    "gray": "#808080",
    "lightgray": "#D3D3D3",
    "black": "#000000"
}

# Create the plot function

def plot_isochrone(geoJSON, lat=None, lon=None, marker_color='blue', isochrone_color=None, map_zoom=13, transport_mode='car'):
    """
    transport_mode: str {'car', 'bicycle', 'person-walking', 'public-transport'}
    """

    # icon_transport_map = {'car': 'car', 'bicycle': 'bicycle', 'foot':'person-walking'}
    
    if isochrone_color is None:
        isochrone_color = FOLIUM_MARKER_COLORS[marker_color]
    else: 
        isochrone_color = FOLIUM_MARKER_COLORS[isochrone_color]

    isochrone_center = geoJSON['features'][0]['properties']['center']
    if lat is None:
        lat = isochrone_center[1]
    if lon is None:
        lon = isochrone_center[0]

    # create the map centered on lat, lon
    map = folium.Map(location=[lat, lon], zoom_start=map_zoom)

    # now to add the isochrone polygon to the map 
    folium.GeoJson(geoJSON, name="Isochrone 30mins driving", color=isochrone_color).add_to(map)
    
    # Inject FA 6 CSS for specific icons
    fa6_link = '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.2/css/all.min.css">'
    map.get_root().html.add_child(folium.Element(fa6_link))

    # Create the marker for the starting point
    folium.Marker(
        location=[lat, lon], 
        tooltip='Starting location', 
        icon=folium.Icon(prefix='fa', icon=transport_mode, color=marker_color)
    ).add_to(map)

    return map


In [8]:
plot_isochrone(isochrone_Edgware, lat_edgware, lon_edgware, marker_color='orange')

In [9]:
fa_v6_potential_icons = ['house-person-leave', 'route', 'bicycle', 'bus', 'car', 'plane', 'train-subway', 'person-walking' ]

# **Geocoding (Use an address to find Latitude and Longitude)**

In [10]:

# load .env and api key incase they aren't already
load_dotenv() 
ORS_API_KEY = os.getenv("ORS_API_KEY")

# enter the address to search 
address = "10 Downing Street, London"


In [11]:
# Creat the function 
# input address
# output [lon, lat]

def find_address_cords(address="16 Upper Hollingdean Road", focus_point=[-0.12574000, 51.50853000]):
    """
    focus_point : the location to focus the address search near
         [lon: float, lat: float] : default [-0.12574000, 51.50853000] (London)
    """
    # Openrouteservice's geocode search api rul
    url = f"https://api.openrouteservice.org/geocode/search"

    params = {
        "api_key": ORS_API_KEY,
        "text":address,
        "size": 1, # number of results to return from search
        "focus.point.lon": focus_point[0],
        "focus.point.lat": focus_point[1]
        # "boudary.country": "GB"           < -- to FORCE GB only results
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    data = response.json()

    coords = data["features"][0]["geometry"]["coordinates"] # [lon, lat] format
    
    return coords

In [12]:
find_address_cords()

[-0.131524, 50.840325]

# **All together**

In [13]:
coords = find_address_cords("Milton Keynes")
isochrone_Hollingbury = get_isochrone(lon=coords[0], lat=coords[1], minutes=45)
plot_isochrone(geoJSON=isochrone_Hollingbury, marker_color='red')